__体素模型数据预处理代码__

In [11]:
%matplotlib widget
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt

#数据读取与质量预处理
data = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_7/iters-200/iters-200-008/topModel_7_200_008.mat')
data = data['data']
data = np.array(data)
print('数据初始维度：',data.shape)
# data = data.transpose(1, 2, 0)
print('数据调整后维度：',data.shape)            #(x,z,y)
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if data[i][j][k] > 0.5:
                data[i][j][k] = 1
            else:
                data[i][j][k] = 0



#数据尺寸扩展为标准尺寸
nelx, nely, nelz = 120, 120, 120
if (nelx - data.shape[0])%2 !=0:
    data = np.pad(data, ((1, 0), (0, 0), (0 ,0)))
if (nelz - data.shape[1])%2 !=0:
    data = np.pad(data, ((0, 0), (1, 0), (0 ,0)))
if (nely - data.shape[2])%2 !=0:
    data = np.pad(data, ((0, 0), (0, 0), (1 ,0)))
data_new = np.pad(data, (((nelx - data.shape[0])//2, (nelx - data.shape[0])//2),
                    ((nely - data.shape[1])//2, (nely - data.shape[1])//2),
                    ((nelz - data.shape[2])//2, (nelx - data.shape[2])//2)))
print('数据标准化后维度：',data_new.shape)



#尺寸标准化后模型可视化
#测试保存
# io.savemat('C:/Users/11573/Desktop/data/standardinizetest.mat',{'standarddata':data_new})
#正式保存
# io.savemat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/standardtopModel_1_100_006.mat',{'standardtest':data_new})

#查找模型存在的单共享边等断裂结构
defect_data = np.zeros((120, 120, 120))
face_voxel = []
search_list = np.zeros((1, 6, 3))
n = 0
t = 0
l = 0
defect_model = np.zeros((nely, nelx, nelz))
defect_index = np.array([[0, 0, 0]])
for i in range(1,(data_new.shape[0]-1)):
    for j in range(1,(data_new.shape[1]-1)):
        for k in range(1,(data_new.shape[2]-1)):
            if data_new[i][j][k] !=0:
                face_voxel.append(data_new[i-1][j][k])
                # print('1:',face_voxel)
                face_voxel.append(data_new[i+1][j][k])
                # print('2:',face_voxel)
                face_voxel.append(data_new[i][j-1][k])
                # print('3:',face_voxel)
                face_voxel.append(data_new[i][j+1][k])
                # print('4:',face_voxel)
                face_voxel.append(data_new[i][j][k-1])
                # print('5:',face_voxel)
                face_voxel.append(data_new[i][j][k+1])
                # print('6:',face_voxel)
                sum_value = sum(face_voxel)
                # print('face_voxel:',face_voxel)
                
                if sum_value == 0 or sum_value == 1:
                    x, y, z = i, j, k
                    defect_array = np.array([[x, y ,z]])
                    # print('i,j,k:',i, j, k)
                    defect_index = np.append(defect_index, defect_array, axis = 0)
                    
                    # defect_index = np.array(defect_index)
                    # print(defect_index.shape)
                    defect_data[i-2:i+2,j-2:j+2,k-2:k+2] = data_new[i-2:i+2,j-2:j+2,k-2:k+2]
                    face_voxel_index = np.array([[[x-1, z, y],
                                                 [x+1, z, y],
                                                 [x, z-1, y],
                                                 [x, z+1, y],
                                                 [x, z, y-1],
                                                 [x, z, y+1]]])
                    search_list = np.append(search_list, face_voxel_index, axis=0)
                    l += 1
                   
                face_voxel = []
                t += 1    
            n += 1

#制作在matlab中显示的缺陷中心点
for t1 in range(1, len(defect_index)):
    exam_index = defect_index[t1]
    x1, y1, z1 = exam_index[0], exam_index[1], exam_index[2]
    defect_model[x1][y1][z1] = 1
defect_data = defect_data + defect_model
# Top3dSTL_v3('C:\Users\11573\Desktop\VSCode_repos\Generate_dataset\Matlab_files\top3d_Model_1\repairtopModel_1.STL',repairdata,'Format','binary','Mode','iso')
#修复模型

# defect_data[69][30][63] = 0
# defect_data[69][30][64] = 0
# defect_data[69][30][65] = 0
# defect_data[69][31][63] = 0
# defect_data[69][31][64] = 0
# defect_data[69][30][54] = 0
# defect_data[69][30][55] = 0
# defect_data[69][31][55] = 0
# defect_data[67][89][59] = 0
# defect_data[67][89][60] = 0
# defect_data[68][88][59] = 0
# defect_data[68][88][60] = 0
# defect_data[68][89][59] = 0
# defect_data[68][89][60] = 0

# defect_data[69][87][59] = 0
# defect_data[69][87][60] = 0
# defect_data[69][88][59] = 0
# defect_data[69][88][60] = 0
# defect_data[69][89][59] = 0
# defect_data[69][89][60] = 0
# defect_data[69][30][55] = 0
# defect_data[69][31][55] = 0
# defect_data[67][89][59] = 0
# defect_data[67][89][60] = 0
# defect_data[68][88][59] = 0
# defect_data[68][88][60] = 0
# defect_data[68][89][59] = 0
# defect_data[68][89][60] = 0
# defect_data[69][87][59] = 0
# defect_data[69][87][60] = 0
# defect_data[69][88][59] = 0
# defect_data[69][88][60] = 0
# defect_data[69][89][59] = 0
# defect_data[69][89][60] = 0
# defect_data[55][61][62] = 0
# defect_data[55][62][62] = 0
# defect_data[67][89][59] = 0
# defect_data[67][89][60] = 0
# defect_data[68][88][59] = 0
# defect_data[68][88][60] = 0
# defect_data[68][89][59] = 0
# defect_data[68][89][60] = 0
# defect_data[69][87][59] = 0
# defect_data[69][87][60] = 0
# defect_data[69][88][59] = 0
# defect_data[69][88][60] = 0
# defect_data[69][89][59] = 0
# defect_data[69][89][60] = 0
# defect_data[69][89][61] = 0
# defect_data[57][78][57] = 0
# defect_data[57][79][57] = 0
# defect_data[63][45][56] = 0
# defect_data[63][46][56] = 0
# defect_data[64][45][56] = 0
# defect_data[64][44][56] = 0
# defect_data[65][43][58] = 0
# defect_data[65][43][59] = 0
# defect_data[65][43][60] = 0
# defect_data[65][43][61] = 0

# defect_data[60][51][53] = 0
# defect_data[61][51][53] = 0
# defect_data[61][50][53] = 0
# defect_data[62][50][53] = 0
# defect_data[62][49][53] = 0
# defect_data[62][48][53] = 0
# defect_data[67][47][62] = 0
# defect_data[68][47][62] = 0
# defect_data[58][48][62] = 0
# defect_data[59][48][62] = 0
# defect_data[65][45][64] = 0
# defect_data[57][47][62] = 0
# defect_data[58][47][62] = 0
# defect_data[57][48][60] = 0
# defect_data[58][50][59] = 0
# defect_data[58][50][60] = 0
# defect_data[59][50][59] = 0
# defect_data[59][50][60] = 0
# defect_data[60][50][59] = 0
# defect_data[60][50][60] = 0
# defect_data[65][43][54] = 0
# defect_data[65][43][55] = 0
# defect_data[65][43][64] = 0
# defect_data[65][43][65] = 0
# defect_data[66][43][54] = 0
# defect_data[66][43][55] = 0
# defect_data[66][43][64] = 0
# defect_data[66][43][65] = 0

np.set_printoptions(threshold=np.inf)
print(defect_index)
print(len(defect_index))
for n in range(defect_data.shape[0]):
    for m in range(defect_data.shape[1]):
        for v in range(defect_data.shape[2]):
            if defect_data[n][m][v] != 0:
                defect_data[n][m][v] = 1


#测试保存
# io.savemat('C:/Users/11573/Desktop/data/defecttopModel.mat',{'defectdata':defect_data})

# #正式保存
io.savemat('../../Matlab_files/top3d_Model/top3d_Model_7/iters-200/iters-200-008/defecttopModel_7_200_008.mat',{'defectdata':defect_data})
# io.savemat('./testdata.mat',{'defectdata':defect_data})
# io.savemat('../../Matlab_files/top3d_Model_1/iters-100/iters-100-006/repairtopModel_7_180_004.mat',{'repairdata':data_new})


数据初始维度： (20, 60, 16)
数据调整后维度： (20, 60, 16)
数据标准化后维度： (120, 120, 120)
[[ 0  0  0]
 [56 61 55]
 [56 61 64]
 [60 70 54]
 [60 70 65]
 [61 65 53]
 [61 65 66]
 [69 30 54]
 [69 30 65]
 [69 57 55]
 [69 57 64]]
11


In [2]:
import scipy.io
import matplotlib.pyplot as plt